In [ ]:
import os
import pandas as pd
from pathlib import Path
import json
from pprint import pprint
from copy import deepcopy
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import warnings
import random
warnings.filterwarnings('ignore')


import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import pipeline


In [ ]:
working_path = Path('/home/zhanj289/side_projects/cord_19/')
data_path = working_path / 'data'

In [ ]:
### data for testing

In [4]:
train_df = pd.read_csv(data_path/ 'cleaned_data'/ 'train_cluster_9.csv')
val_df = pd.read_csv(data_path/ 'cleaned_data'/  'dev_cluster_9.csv')

In [5]:
## seprate some from training to have validation
test_df_index = random.sample(list(range(train_df.shape[0])), train_df.shape[0]//5)

test_df = train_df.iloc[test_df_index].reset_index(drop = True)

train_df =  train_df.drop(test_df_index).reset_index(drop = True)

In [6]:
## remove stop words
# def remove_stopwords(list_of_string):
#     stop_words = set(stopwords.words('english')) 
#     return [w for w in list_of_string if not w in stop_words] 
  

In [7]:
## remove very short words, like single letter

# def remove_short_words(tokens, min_len):
#     return [w for w in tokens if len(w)>=min_len] 


In [8]:
## remove the word 'abstract'
def remove_specific_word(text, word_list):
    text_word = text.split()
    new_text = [w for w in text_word if w not in word_list]
    new_text = " ".join(new_text)
    
    return new_text

In [9]:
train_df['title'] = train_df['title'].apply(lambda x: remove_specific_word(x, ['title']))

In [10]:
train_df['abstract'] = train_df['abstract'].apply(lambda x: remove_specific_word(x, ['abstract']))

In [11]:
test_df['title'] = test_df['title'].apply(lambda x: remove_specific_word(x, ['title']))

In [12]:
test_df['abstract'] = test_df['abstract'].apply(lambda x: remove_specific_word(x, ['abstract']))

In [13]:
val_df['title'] = val_df['title'].apply(lambda x: remove_specific_word(x, ['title']))

In [14]:
val_df['abstract'] = val_df['abstract'].apply(lambda x: remove_specific_word(x, ['abstract']))

In [15]:
test_df.shape

(587, 3)

In [16]:
val_df.shape

(695, 3)

In [17]:
train_df.shape

(2349, 3)

In [18]:
## output data

In [19]:
output_path = working_path / 'summarization' / 'bart' / 'cluster_9_train_test_data'

In [20]:
output_path

PosixPath('/home/zhanj289/side_projects/cord_19/summarization/bart/cluster_9_train_test_data')

In [21]:
lines =list(train_df['title'])
with open(output_path / 'train.target', 'w') as f:
    f.writelines('\n'.join(lines))

In [22]:
lines =list(train_df['abstract'])
with open(output_path/ 'train.source', 'w') as f:
    f.writelines('\n'.join(lines))

In [23]:
lines =list(val_df['title'])
with open(output_path/ 'val.target', 'w') as f:
    f.writelines('\n'.join(lines))

In [24]:
lines =list(val_df['abstract'])
with open(output_path/ 'val.source', 'w') as f:
    f.writelines('\n'.join(lines))

In [25]:
lines =list(test_df['title'])
with open(output_path/ 'test.target', 'w') as f:
    f.writelines('\n'.join(lines))

In [26]:
lines =list(test_df['abstract'])
with open(output_path/ 'test.source', 'w') as f:
    f.writelines('\n'.join(lines))

In [27]:
examples = []
with open(output_path/ 'train.source', "r") as f:
    for text in f.readlines():
        examples.append(text)

In [29]:
len(examples)

2349

### All data

In [258]:
all_df = pd.read_csv(data_path/ 'cleaned_data'/ 'train_dataset.csv')


In [259]:
all_df.shape

(40430, 14)

In [289]:
## seprate some from training to have validation

df_index = list(range(all_df.shape[0]))
random.shuffle(df_index)

In [290]:
## 60-20-20 split
val_index, test_index, train_index = df_index[0: len(df_index)//5], df_index[len(df_index)//5:2*len(df_index)//5 ], df_index[2*len(df_index)//5:]

In [291]:
len(val_index)

8086

In [292]:
len(test_index)

8086

In [293]:
len(train_index)

24258

In [294]:
## sepearate
test_df = all_df.iloc[test_index,:].reset_index(drop = True)

train_df =  all_df.iloc[train_index,:].reset_index(drop = True)

val_df =  all_df.iloc[val_index,:].reset_index(drop = True)

In [297]:
test_df.shape

(8086, 14)

In [298]:
## remove the word 'abstract'
def remove_specific_word(text, word_list):
    text_word = text.split()
    new_text = [w for w in text_word if w not in word_list]
    new_text = " ".join(new_text)
    
    return new_text

In [299]:
train_df['title'] = train_df['title'].apply(lambda x: remove_specific_word(x, ['title']))

In [300]:
train_df['abstract'] = train_df['abstract'].apply(lambda x: remove_specific_word(x, ['abstract']))

In [301]:
test_df['title'] = test_df['title'].apply(lambda x: remove_specific_word(x, ['title']))

In [302]:
test_df['abstract'] = test_df['abstract'].apply(lambda x: remove_specific_word(x, ['abstract']))

In [303]:
val_df['title'] = val_df['title'].apply(lambda x: remove_specific_word(x, ['title']))

In [304]:
val_df['abstract'] = val_df['abstract'].apply(lambda x: remove_specific_word(x, ['abstract']))

In [305]:
test_df.shape

(8086, 14)

In [306]:
val_df.shape

(8086, 14)

In [307]:
train_df.shape

(24258, 14)

In [ ]:
## output data

In [311]:
output_path = working_path / 'summarization' / 'bart' / 'all_train_test_data'

In [312]:
output_path

PosixPath('/home/zhanj289/side_projects/cord_19/summarization/bart/all_train_test_data')

In [343]:
lines =list(train_df['title'])
with open(output_path / 'train.target', 'w') as f:
    f.writelines('\n'.join(lines))

In [344]:
lines =list(train_df['abstract'])
with open(output_path/ 'train.source', 'w') as f:
    f.writelines('\n'.join(lines))

In [345]:
lines =list(val_df['title'])
with open(output_path/ 'val.target', 'w') as f:
    f.writelines('\n'.join(lines))

In [346]:
lines =list(val_df['abstract'])
with open(output_path/ 'val.source', 'w') as f:
    f.writelines('\n'.join(lines))

In [347]:
lines =list(test_df['title'])
with open(output_path/ 'test.target', 'w') as f:
    f.writelines('\n'.join(lines))

In [348]:
lines =list(test_df['abstract'])
with open(output_path/ 'test.source', 'w') as f:
    f.writelines('\n'.join(lines))

In [349]:
len(lines)

8086

In [356]:
examples = []
with open(output_path/ 'train.source', "r") as f:
    for text in f.readlines():
        examples.append(text)

In [357]:
len(examples)

24258

In [ ]:
### test bart

In [18]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(torch_device)

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

summarizer = pipeline(task="summarization", model=model, tokenizer=tokenizer)

cpu


[{'summary_text': 'Bats are natural reservoirs of many pathogens few studies have been conducted on the genetic variation and detection of selection in major histocompatibility complex mhc genes. These genes are critical for resistance and susceptibility to diseases and host-path'}]

In [92]:
train_df['title'][2]

'spatial pattern of genetic diversity and selection in the mhc class ii drb of three neotropical bat species'

In [ ]:
### Rouge Score

In [89]:
from rouge_score import rouge_scorer
from bleu import list_bleu

# rogue score


#len(train_df['abstract'])

In [136]:
def eval_rouge(df, truth_y_col, train_text_col):

    rouge_score_list = []
    generated_title_list= []
    
    for i in range(df.shape[0]):

        # obtain 
        ref = train_df[truth_y_col][i]
        title_length = len(ref)

        ## obtain generated title
        generated_title = summarizer(train_df[train_text_col][i],
                                     min_length=4, 
                                     max_length=50)
        
        generated_title = generated_title[0]['summary_text']
        
        generated_title_list.append(generated_title)

        # rogue score
        scorer = rouge_scorer.RougeScorer(['rouge1','rougeL'], 
                                          use_stemmer=True)

        rouge_scores = scorer.score(ref, generated_title)


        rouge1 = round(rouge_scores['rouge1'][2]*100, 2)

        rougeL = round(rouge_scores['rougeL'][2]*100, 2)

        rouge_score_list.append((rouge1, rougeL))
    
    df['generated_text'] = generated_title_list

    # calculate average score and store
    score_dict = {}

    score_dict['rogue-1'], score_dict['rogue-L']  = list(map(lambda y: round(sum(y) / float(len(y)),2), zip(*rouge_score_list)))

    
    return score_dict, df

In [138]:
scores, testtesttest = eval_rouge(testtest, 'title', 'abstract')

In [139]:
scores

{'rogue-1': 31.64, 'rogue-L': 21.07}

In [ ]:
scores

In [140]:
testtesttest

,paper_id,title,abstract,generated_text
0,4ba8032f2aedc74eda0d7f73e7b4977ad78a977d,establishment of canine and feline cells expre...,signaling lymphocyte activation molecule slam ...,signaling lymphocyte activation molecule slam...
1,8f78b051ab35efcfb25e72166844b19f8c886bc9,retrospective biomolecular investigation of co...,a b s t r a c t abortion and neonatal mortalit...,Up to 55% and 33% of these cases remain withou...
2,73d9db430955cb7c45280e5e29981795c674848f,spatial pattern of genetic diversity and selec...,background although bats are natural reservoir...,Bats are natural reservoirs of many pathogens ...
3,37b045030f0700ae3ea09514c1e929cab8489cd3,systematic implications of seed coat morpholog...,seeds from 37 species of nama hydrophyllaceae ...,Seeds from 37 species of nama hydrophyllaceae ...
4,4b8b3a42d14171561329c896abf4c1d5f4fb7857,dangerous viral pathogens of animal origin ris...,most of emerging infectious diseases affecting...,Most emerging infectious diseases affecting hu...


In [141]:
score_dict

{'rogue-1': 31.64, 'rogue-L': 21.07}

In [97]:
scores['rouge1']

Score(precision=0.23076923076923078, recall=0.5, fmeasure=0.3157894736842105)

In [146]:
testtest = []
with open(working_path/'summarization/bart/cnn_tiny/test.source', "r") as f:
        for text in f.readlines():
            testtest.append(text)

In [148]:
len(testtest)

4

In [149]:
testtesty = []
with open(working_path/'summarization/bart/cnn_tiny/test.target', "r") as f:
        for text in f.readlines():
            testtesty.append(text)